# Saving model architecture only

In this reading you will learn how to save a model's architecture, but not its weights.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import json
import numpy as np

2025-11-27 01:48:03.868667: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 01:48:03.907753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764229683.931446  157374 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764229683.940720  157374 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764229683.960526  157374 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In previous videos and notebooks you have have learned how to save a model's weights, as well as the entire model - weights and architecture.

### Accessing a model's configuration
A model's *configuration* refers to its architecture. TensorFlow has a convenient way to retrieve a model's architecture as a dictionary. We start by creating a simple fully connected feedforward neural network with 1 hidden layer.

In [2]:
model = Sequential([
    Dense(units=32, input_shape=(32, 32, 3), activation='relu', name='dense_1'),
    Dense(units=10, activation='softmax', name='dense_2')
])

/home/nvidia/miniforge3/envs/rapids-25.10-cuda12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1764229692.760654  157374 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


A TensorFlow model has an inbuilt method `get_config` which returns the model's architecture as a dictionary:

In [3]:
config_dict = model.get_config()
print(config_dict)

{'name': 'sequential', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 32, 32, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Dense', 'config': {'name': 'dense_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 32, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 

### Creating a new model from the config
A new TensorFlow model can be created from this config dictionary. This model will have reinitialized weights, which are not the same as the original model.

In [4]:
model_same_config = tf.keras.Sequential.from_config(config_dict)

We can check explicitly that the config of both models is the same, but the weights are not:

In [5]:
print('Same config:',
      model.get_config() == model_same_config.get_config())
print('Same value for first weight matrix:',
      np.allclose(model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: True
Same value for first weight matrix: False


For models that are not `Sequential` models, use `tf.keras.Model.from_config` instead of `tf.keras.Sequential.from_config`.

### Other file formats: JSON and YAML
It is also possible to obtain a model's config in JSON or YAML formats. This follows the same pattern:

In [6]:
json_string = model.to_json()
print(json_string)

{"module": "keras", "class_name": "Sequential", "config": {"name": "sequential", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 32, 32, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer"}, "registered_name": null}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": n

The JSON format can easily be written out and saved as a file:

In [7]:
# Write out JSON config file

path = '05_only_architecture/json_config.json'

# Create directory if it doesn't exist
import os
os.makedirs(os.path.dirname(path), exist_ok=True)

with open(path, 'w') as f:
    json.dump(json_string, f)
del json_string

In [9]:
# Read in JSON config file again

with open(path, 'r') as f:
    json_string = json.load(f)

In [10]:
# Reinitialize model

model_same_config = tf.keras.models.model_from_json(json_string)

In [11]:
# Same config, new weights

print('Same config:',
      model.get_config() == model_same_config.get_config())
print('Same value for first weight matrix:',
      np.allclose(model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: False
Same value for first weight matrix: False


The YAML format is similar. The details of writing out YAML files, loading them and using them to create a new model are similar as for the JSON files, so we won't show it here.

In [12]:
yaml_string = model.to_yaml()
print(yaml_string)

AttributeError: 'Sequential' object has no attribute 'to_yaml'

Writing out, reading in and using YAML files to create models is similar to JSON files.

### Further reading and resources
* https://www.tensorflow.org/guide/keras/save_and_serialize#architecture-only_saving
* https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model